### Домашнее задание к вебинару 1 - Предобработка текста

**Постановка задачи**. Осуществить предобработку данных с Твиттера, чтобы отчищенный данные в дальнейшем использовать для задачи классификации. Данный датасет содержит негативные (label = 1) и нейтральные (label = 0) высказывания.
Для работы объединим train_df и test_df.

In [248]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
import os

In [249]:
# Load train data 

train_df = pd.read_csv('train_tweets.csv')
train_df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [250]:
train_df.shape

(31962, 3)

In [251]:
# Load test data

test_df = pd.read_csv('test_tweets.csv')
test_df.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [252]:
test_df.shape

(17197, 2)

In [253]:
# Make union dataset

tweet_df = train_df.append(test_df, ignore_index = True, sort = False)
tweet_df.head()

,id,label,tweet
0,1,0.0,@user when a father is dysfunctional and is s...
1,2,0.0,@user @user thanks for #lyft credit i can't us...
2,3,0.0,bihday your majesty
3,4,0.0,#model i love u take with u all the time in ...
4,5,0.0,factsguide: society now #motivation


In [254]:
print(combined_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49159 entries, 0 to 49158
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    49159 non-null  int64  
 1   label                 31962 non-null  float64
 2   tweet                 49159 non-null  object 
 3   tweet_token           49159 non-null  object 
 4   tweet_token_filtered  49159 non-null  object 
 5   tweet_stemmed         49159 non-null  object 
 6   tweet_lemmatized      49159 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 2.6+ MB
None


**Задание 1.** Удалить @user из всех твитов

In [255]:
tweet_df.head(2)

,id,label,tweet
0,1,0.0,@user when a father is dysfunctional and is s...
1,2,0.0,@user @user thanks for #lyft credit i can't us...


In [256]:
def delete_user_mark(string):  
    try:
        user_mark = re.findall("@[\w]*", string)[0]
        return re.sub(user_mark, ' ',  string)
    except:
        return string

In [257]:
tweet_df['tweet'] = tweet_df['tweet'].apply(lambda x: delete_user_mark(x))

In [258]:
tweet_df.head(2)

,id,label,tweet
0,1,0.0,when a father is dysfunctional and is so se...
1,2,0.0,thanks for #lyft credit i can't use cause ...


**Задание 2.** Изменить регистр твитов на нижний с помощью .lower()

In [259]:
tweet_df.tweet = tweet_df.tweet.apply(lambda x: x.lower())

In [260]:
tweet_df.head(3)

,id,label,tweet
0,1,0.0,when a father is dysfunctional and is so se...
1,2,0.0,thanks for #lyft credit i can't use cause ...
2,3,0.0,bihday your majesty


**Задание 3.** Заменить сокращения с апострофами (пример: ain't, can't), используя apostrophe_dict. 

In [261]:
apostrophe_dict = {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

In [262]:
def transform_text(string, dict_for_sub):
    string_splited = string.split(' ')
    for word in set(string_splited):
        if word in dict_for_sub.keys():
            string = re.sub(re.escape(word), dict_for_sub[word], string)
        else:
            continue
    return string

In [263]:
# def transform_text(string, dict_for_transform):
#     for i, word in enumerate(string.split(' ')):
#         for key in dict_for_transform.keys():
#             if word == key:
#                 string = string.replace(word, dict_for_transform[key])   
#             else:
#                 continue
#     return string

In [264]:
tweet_df['tweet'] = tweet_df['tweet'].apply(lambda x: transform_text(x, apostrophe_dict))

In [265]:
tweet_df['tweet'].head(5)

0       when a father is dysfunctional and is so se...
1        thanks for #lyft credit i cannot use cause...
2                                  bihday your majesty
3    #model   i love u take with u all the time in ...
4               factsguide: society now    #motivation
Name: tweet, dtype: object

**Задание 4.** Заменить сокращения на их полные формы, используя short_word_dict.

In [266]:
short_word_dict = {
"121": "one to one",
"a/s/l": "age, sex, location",
"adn": "any day now",
"afaik": "as far as I know",
"afk": "away from keyboard",
"aight": "alright",
"alol": "actually laughing out loud",
"b4": "before",
"b4n": "bye for now",
"bak": "back at the keyboard",
"bf": "boyfriend",
"bff": "best friends forever",
"bfn": "bye for now",
"bg": "big grin",
"bta": "but then again",
"btw": "by the way",
"cid": "crying in disgrace",
"cnp": "continued in my next post",
"cp": "chat post",
"cu": "see you",
"cul": "see you later",
"cul8r": "see you later",
"cya": "bye",
"cyo": "see you online",
"dbau": "doing business as usual",
"fud": "fear, uncertainty, and doubt",
"fwiw": "for what it's worth",
"fyi": "for your information",
"g": "grin",
"g2g": "got to go",
"ga": "go ahead",
"gal": "get a life",
"gf": "girlfriend",
"gfn": "gone for now",
"gmbo": "giggling my butt off",
"gmta": "great minds think alike",
"h8": "hate",
"hagn": "have a good night",
"hdop": "help delete online predators",
"hhis": "hanging head in shame",
"iac": "in any case",
"ianal": "I am not a lawyer",
"ic": "I see",
"idk": "I don't know",
"imao": "in my arrogant opinion",
"imnsho": "in my not so humble opinion",
"imo": "in my opinion",
"iow": "in other words",
"ipn": "I’m posting naked",
"irl": "in real life",
"jk": "just kidding",
"l8r": "later",
"ld": "later, dude",
"ldr": "long distance relationship",
"llta": "lots and lots of thunderous applause",
"lmao": "laugh my ass off",
"lmirl": "let's meet in real life",
"lol": "laugh out loud",
"ltr": "longterm relationship",
"lulab": "love you like a brother",
"lulas": "love you like a sister",
"luv": "love",
"m/f": "male or female",
"m8": "mate",
"milf": "mother I would like to fuck",
"oll": "online love",
"omg": "oh my god",
"otoh": "on the other hand",
"pir": "parent in room",
"ppl": "people",
"r": "are",
"rofl": "roll on the floor laughing",
"rpg": "role playing games",
"ru": "are you",
"shid": "slaps head in disgust",
"somy": "sick of me yet",
"sot": "short of time",
"thanx": "thanks",
"thx": "thanks",
"ttyl": "talk to you later",
"u": "you",
"ur": "you are",
"uw": "you’re welcome",
"wb": "welcome back",
"wfm": "works for me",
"wibni": "wouldn't it be nice if",
"wtf": "what the fuck",
"wtg": "way to go",
"wtgp": "want to go private",
"ym": "young man",
"gr8": "great"
}

In [267]:
tweet_df['tweet'] = tweet_df['tweet'].apply(lambda x: transform_text(x, short_word_dict))

In [268]:
tweet_df['tweet']

0           when a father is dysfunctional and is so se...
1            thanks for #lyft credit i cannot use cause...
2                                      bihday your majesty
3        #model   i love you take with you all the time...
4                   factsguide: society now    #motivation
                               ...                        
49154    thought factory: left-right polarisation! #tru...
49155    feeling like a mermaid ð #hairflip #neverre...
49156    #hillary #campaigned today in #ohio((omg)) &am...
49157    happy, at work conference: right mindset leads...
49158    my   song "so glad" free download!  #shoegaze ...
Name: tweet, Length: 49159, dtype: object

**Задание 5.** Заменить эмотиконы (пример: ":)" = "happy"), используя emoticon_dict.

In [269]:
emoticon_dict = {
":)": "happy",
":‑)": "happy",
":-]": "happy",
":-3": "happy",
":->": "happy",
"8-)": "happy",
":-}": "happy",
":o)": "happy",
":c)": "happy",
":^)": "happy",
"=]": "happy",
"=)": "happy",
"<3": "happy",
":-(": "sad",
":(": "sad",
":c": "sad",
":<": "sad",
":[": "sad",
">:[": "sad",
":{": "sad",
">:(": "sad",
":-c": "sad",
":-< ": "sad",
":-[": "sad",
":-||": "sad"
}

In [270]:
tweet_df['tweet'] = tweet_df['tweet'].apply(lambda x: transform_text(x, emoticon_dict))

In [271]:
tweet_df['tweet'].head(4)

0       when a father is dysfunctional and is so se...
1        thanks for #lyft credit i cannot use cause...
2                                  bihday your majesty
3    #model   i love you take with you all the time...
Name: tweet, dtype: object

**Задание 6.** Заменить пунктуацию на пробелы, используя re.sub() и паттерн r'[^\w\s]'

In [272]:
def clean_punkt(string):
    try:
        return re.sub("[^\w\s]", ' ',  string)

    except:
        return string

In [273]:
tweet_df['tweet'] = tweet_df['tweet'].apply(lambda x: clean_punkt(x))

In [274]:
tweet_df['tweet'].head(4)

0       when a father is dysfunctional and is so se...
1        thanks for  lyft credit i cannot use cause...
2                                  bihday your majesty
3     model   i love you take with you all the time...
Name: tweet, dtype: object

**Задание 7.** Заменим спец. символы на пробелы, используя re.sub() и паттерн r'[^a-zA-Z0-9]'

In [275]:
def clean_spec_symbols(string):
    try:
        return re.sub("[^a-zA-Z0-9]", ' ',  string)

    except:
        return string

In [276]:
tweet_df['tweet'] = tweet_df['tweet'].apply(lambda x: clean_spec_symbols(x))

**Задание 8.** Заменить числа на пробелы, используя re.sub() и паттерн r'[^a-zA-Z]'

In [277]:
def clean_digits(string):
    try:
        return re.sub("[^a-zA-Z]", ' ',  string)

    except:
        return string

In [278]:
tweet_df['tweet'] = tweet_df['tweet'].apply(lambda x: clean_digits(x))

**Задание 9.** Удалить из текста слова длиной в 1 символ, используя ' '.join([w for w in x.split() if len(w)>1])

In [279]:
def clean_one_lenght_words(string):
    return ' '.join([word for word in string.split()if len(word) > 1])

In [280]:
tweet_df['tweet'] = tweet_df['tweet'].apply(lambda x: clean_one_lenght_words(x))

**Задание 10.** Поделить твиты на токены с помощью nltk.tokenize.word_tokenize, создав новый столбец 'tweet_token'.

In [281]:
from nltk.tokenize import word_tokenize as w_tknz

In [282]:
tweet_df['tweet_token'] = tweet_df['tweet'].apply(w_tknz)

In [283]:
tweet_df['tweet_token']

0        [when, father, is, dysfunctional, and, is, so,...
1        [thanks, for, lyft, credit, can, not, use, cau...
2                                  [bihday, your, majesty]
3        [model, love, you, take, with, you, all, the, ...
4                   [factsguide, society, now, motivation]
                               ...                        
49154    [thought, factory, left, right, polarisation, ...
49155    [feeling, like, mermaid, hairflip, neverready,...
49156    [hillary, campaigned, today, in, ohio, omg, am...
49157    [happy, at, work, conference, right, mindset, ...
49158    [my, song, so, glad, free, download, shoegaze,...
Name: tweet_token, Length: 49159, dtype: object

**Задание 11.** Удалить стоп-слова из токенов, используя nltk.corpus.stopwords. Создать столбец 'tweet_token_filtered' без стоп-слов.

In [284]:
from nltk.corpus import stopwords

In [285]:
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [286]:
tweet_df['tweet_token_filtered'] = [[word for word in tweet if word not in set(stopwords.words('english'))]  \
                                                     for tweet in combined_df['tweet_token']]

In [287]:
tweet_df['tweet_token_filtered'].head(3)

0    [father, dysfunctional, selfish, drags, kids, ...
1    [thanks, lyft, credit, use, cause, offer, whee...
2                                    [bihday, majesty]
Name: tweet_token_filtered, dtype: object

**Задание 12.** Применить стемминг к токенам с помощью nltk.stem.PorterStemmer. Создать столбец 'tweet_stemmed' после применения стемминга.

In [288]:
from nltk.stem import PorterStemmer

In [289]:
stemmer = PorterStemmer()

In [290]:
# Example
stemmer.stem('speaking')

'speak'

In [291]:
tweet_df['tweet_stemmed'] = [[stemmer.stem(word) for word in sentence] for sentence in tweet_df['tweet_token_filtered']]

**Задание 13.** Применить лемматизацию к токенам с помощью nltk.stem.wordnet.WordNetLemmatizer. Создадим столбец 'tweet_lemmatized' после применения лемматизации.

In [292]:
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/aleksandrinavatkina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [293]:
lemmatizer = WordNetLemmatizer()

# Example
lemmatizer.lemmatize(word = 'swam', pos = 'v')

'swim'

In [294]:
tweet_df['tweet_lemmatized'] = [[stemmer.stem(word) for word in sentence] for sentence in tweet_df['tweet_token_filtered']]

In [295]:
tweet_df.head(3)

,id,label,tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunct, selfish, drag, kid, dysfunc..."
1,2,0.0,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thank, lyft, credit, use, caus, offer, wheelc..."
2,3,0.0,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesti]"


**Задание 14.** Сохранить результат предобработки в pickle-файл.

In [296]:
import pickle

In [298]:
with open('data_tweet_preprocessed.pkl.pickle', 'wb') as file:
    pickle.dump(tweet_df, file)